## About the dataset 

The dataset for this task is built using articles and headline pairs from several leading newspapers of the country. 

The task is to generate a meaningful fixed length summary for each article. 

In [73]:
hindi_train = '/kaggle/input/hindidataset/hindi_train.csv'
hindi_test = '/kaggle/input/hindidataset/HindiNews_test.csv'

In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader
from transformers import(
    AutoTokenizer as myTokenizer,
    AutoModelForSeq2SeqLM
)

import pytorch_lightning as pl

In [75]:
train = hindi_train
test = hindi_test

In [76]:
df = pd.read_csv(train)
dftest = pd.read_csv(test)

In [77]:
df = df[["Summary", "Article"]]
# only preserve needed columns

In [78]:
df.head(3)

,Summary,Article
0,Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...


In [79]:
dftest.head(3)

,id,Article,Heading
0,HindiNews_test_0,‘वारिस पंजाब दे’ संगठन के चीफ अमृतपाल सिंह को ...,भिंडरांवाले के गांव में वारिस पंजाब दे का चीफ ...
1,HindiNews_test_1,कांग्रेस नेता राहुल गांधी जल्द ही साउथ दिल्ली ...,सांसदी जाने के बाद अप्रैल में सरकारी बंगला छोड़...
2,HindiNews_test_2,संसद परिसर में मंगलवार को आम आदमी पार्टी (AAP)...,"BJP का तंज- झूठ बोले कौआ काटे, राघव का पलटवार-..."


In [80]:
df_train, df_valid = train_test_split(df, random_state = 42, test_size = 0.1)
df_train.shape, df_valid.shape

((19102, 2), (2123, 2))

## Create a Custom Dataset Object
In PyTorch, this is done by subclassing a torch.utils.data.Dataset object and implementing \_\_len__ and \_\_getitem__

For parameters to pass to T5 tokenizer, refer: 

https://huggingface.co/docs/transformers/v4.22.2/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast

T5 tokenizer inherits from PreTrainedTokenizerFast which contains most of the main methods. 

For pytorch:

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [81]:
import re
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [82]:
class NewsSummaryDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: myTokenizer,
        text_max_tokens_len: int = 512,
        summary_max_tokens_len: int = 75
    ):
        self.data = data
        self.tokenizer = tokenizer
        self.text_max_tokens_len = text_max_tokens_len
        self.summary_max_tokens_len = summary_max_tokens_len

    def __getitem__(self, index):
        data_row = self.data.iloc[index]
        article, summary = data_row["Article"], data_row["Summary"]
        text_encoding = self.tokenizer(
            WHITESPACE_HANDLER(article),
            max_length = self.text_max_tokens_len,
            truncation = True,
            padding= "max_length", # all sents padded to max_length
            return_tensors = "pt" # pytorch tensors
        )
        summary_encoding = self.tokenizer(
            WHITESPACE_HANDLER(summary),
            max_length = self.summary_max_tokens_len,
            truncation = True,
            padding= "max_length", # all sents padded to max_length
            return_tensors = "pt" #pytorch tensors
        )
        # actual labels
        labels = summary_encoding["input_ids"]
        #Note: the input_ids includes padding too, so replace pad tokens(zero value) with value of -100
        labels[labels==0] = -100

        return {
            "text": article,
            "summary": summary,
            "text_input_ids": text_encoding["input_ids"].flatten(),
            "text_attention_mask":text_encoding["attention_mask"].flatten(),
            # attention mask: tells us what tokens to pay attention to, value: 0/1
            "labels": labels.flatten(),
            "labels_attention_mask": summary_encoding["attention_mask"].flatten(),
        }

    def __len__(self):
        return len(self.data)

In [83]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("saliq7/GUJU_MT5")

In [84]:
class NewsSummaryDatasetModule(pl.LightningModule):
    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: myTokenizer,
        batch_size: int = 8,
        text_max_tokens_len: int = 512,
        summary_max_tokens_len: int = 75
    ):
        super().__init__()
        self.train_df = train_df
        self.test_df = test_df
        
        self.bs = batch_size
        self.tokenizer = tokenizer
        self.text_max_tokens_len = text_max_tokens_len
        self.summary_max_tokens_len = summary_max_tokens_len
        self.setup()
    
    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_tokens_len,
            self.summary_max_tokens_len
            )
        self.test_dataset = NewsSummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_tokens_len,
            self.summary_max_tokens_len
            )
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size = self.bs,
            shuffle = True,
            num_workers = 7
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.bs,
            shuffle = False,
            num_workers = 7
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size = self.bs,
            shuffle = False,
            num_workers = 7
        )    
    

In [85]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text2text-generation", model="ai4bharat/IndicBART")

In [86]:
# df['Article'] = df['Article'].apply(lambda x: x[:1024])

In [87]:
# pipe(df['Article'][0], max_length=75), df['Summary'][0]

In [88]:
# from rouge import Rouge
# scorer = Rouge()


In [89]:
MODEL_NAME = "google/mt5-base"

TOKENIZER_NAME = "google/mt5-base"
tokenizer = myTokenizer.from_pretrained(TOKENIZER_NAME)

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Initialize the data module

In [90]:
# Setting up parameters
BATCH_SIZE = 4
EPOCHS = 0

In [91]:
datamodule = NewsSummaryDatasetModule(
    df_train,
    df_valid,
    tokenizer,
    batch_size = BATCH_SIZE
)

## MODEL 


In [92]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, return_dict = True)

    def forward(
        self,
        input_ids,
        attention_mask,
        decoder_attention_mask,
        labels        
    ):
        # calls the model, actual place where model structure can be defined
        # only 2 inputs are required for the model in order to compute a loss: input_ids and labels
        output = self.model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            labels=labels,
            decoder_attention_mask = decoder_attention_mask
        )
        
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx ):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        # call forward method
        loss, logits = self(
            input_ids,
            attention_mask,
            labels,
            labels_attention_mask
        )
        
        # https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#log
        # Log a key, value pair.
        self.log("training_loss", loss)
        
        return loss
        
    
    def validation_step(self, batch, batch_idx ):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        # call forward method
        loss, logits = self(
            input_ids,
            attention_mask,
            labels,
            labels_attention_mask
        )
        self.log("validation_loss", loss)
        return loss
    
    def test_step(self, batch, batch_idx ):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        # call forward method
        loss, logits = self(
            input_ids,
            attention_mask,
            labels,
            labels_attention_mask
        )
        
        self.log("test_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.0001)
        return optimizer

Initialize model

In [93]:
model = NewsSummaryModel()

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [23]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard
# %tensorboard --logdir ./lightning_logs

## Checkpointing
- For basic checkpointing,
    - Saving: save training model at every epoch: trainer = Trainer(default_root_dir="some/path/")
    - Loading: checkpoint = torch.load(CKPT_PATH)
    - Disable: trainer = Trainer(enable_checkpointing=False)
- For fine-grained control over checkpointing behavior, use the ModelCheckpoint object
    - Save the model periodically by monitoring a quantity. Every metric logged with log() or log_dict() in LightningModule is a candidate for the monitor key
    - https://pytorch-lightning.readthedocs.io/en/stable/common/checkpointing_intermediate.html
   

In [94]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath="output/checkpoints",
    filename="best_checkpoint",
    save_top_k=1,
    mode = 'min',
    monitor="validation_loss",
    verbose = True
)

## Logging
- Log to local file system in TensorBoard format.
- Logs are saved to os.path.join(save_dir, name, version)

https://pytorch-lightning.readthedocs.io/en/stable/extensions/generated/pytorch_lightning.loggers.TensorBoardLogger.html

In [25]:
# logger = pl.loggers.TensorBoardLogger("lightning_logs", name="lang_summarization")

## TRAINING
Ligtning offers various modes for trainer by setting flags.
https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html#trainer-class-api

In [95]:
trainer = pl.Trainer(
#     logger = logger,
    callbacks = [checkpoint_callback],
    max_epochs = EPOCHS,
#     accelerator='gpu', 
    devices=1,
    num_sanity_val_steps=0
    
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> Trainer.fit(model, train_dataloaders=None, val_dataloaders=None, datamodule=None, ckpt_path=None)

We don't need to specify train/val dataloaders instead we can directly pass LightningDataModule which alread has all the dataloaders.

In [96]:
trainer.fit(model, datamodule = datamodule)

In [60]:
trainer.validate(model, dataloaders=datamodule.val_dataloader())
# checking purposes only

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Process Process-6:
Process Process-8:
Process Process-5:
Process Process-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/opt/conda/lib/python3.7/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/opt/conda/lib/python3.7/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizer

### Load from checkpoint

In [97]:
trainer.save_checkpoint('checkpoints/best_checkpoint.ckpt')

In [98]:
trained_model = NewsSummaryModel.load_from_checkpoint('/kaggle/working/checkpoints/best_checkpoint.ckpt')

In [99]:
trained_model.freeze() # speeds up inference

## INFERENCE

In [100]:
def summarizer(text):
    text_encoding = tokenizer(
        text,
        max_length = 512,
        return_attention_mask = True,
        truncation = True,
        add_special_tokens = True,
        padding= "max_length", # all sents padded to max_length
        return_tensors = "pt" #pytorch tensors
        )
    
    generated_ids = trained_model.model.generate( 
        # need to do model.model because "model" is actually a class "NewsSummaryModel" having attribute self.model as T5 
        input_ids = text_encoding["input_ids"],
        attention_mask = text_encoding["attention_mask"],
        max_length = 75,
        num_beams = 2,
        early_stopping = True
    )
    summary = tokenizer.batch_decode(generated_ids,skip_special_tokens = True)
    # print(summary)
    return summary

In [101]:
text = df_valid.iloc[10]["Article"]
summary = df_valid.iloc[10]["Summary"]
text, summary

('नई दिल्ली। कांग्रेस ने शनिवार (29 अगस्त) को एक बार फिर नरेंद्र मोदी सरकार पर फेसबुक के स्वामित्व वाले व्हाट्सएप को लेकर जमकर हमला बोला। कांग्रेस ने कहा कि कथित रूप से भारतीय जनता पार्टी (भाजपा) द्वारा अप्रत्यक्ष तरीके से नियंत्रित किया जा रहा है। कांग्रेस ने इस मुद्दे पर संयुक्त संसदीय समिति (जेपीसी) की जांच की मांग भी की है। कांग्रेस ने यह भी मांग की कि जांच पूरी न होने तक व्हाट्सएप को अपनी भुगतान (पेमेंट) सेवाएं शुरू करने की अनुमति नहीं दी जानी चाहिए।BSF ने बॉर्डर के पास पकड़ी 20 फीट लंबी सुरंग, पाकिस्तान में बने बोरियों से ढका था मुंहकांग्रेस प्रवक्ता पवन खेड़ा ने यहां एक संवाददाता सम्मेलन को संबोधित करते हुए कहा, "सोशल मीडिया दिग्गज फेसबुक के भारतीय संचालन और नरेंद्र मोदी के नेतृत्व वाली भाजपा सरकार के बीच अपवित्र घनिष्ठता अब तेजी से उभर रही है और अब ये केवल भारत का नहीं, बल्कि एक वैश्विक मुद्दा है।" एक अंतर्राष्ट्रीय प्रकाशन (पब्लिकेशन) में प्रकाशित एक मीडिया रिपोर्ट का हवाला देते हुए कांग्रेस नेता ने कहा, "एक प्रतिष्ठित वैश्विक प्रकाशन ने खुलासा किया है कि व्हाट्सएप, जिसका उपयो

In [102]:
summarizer(text)

['<extra_id_0> सत्ताधारी कांग्रेस प्रवक्ता']

In [104]:
actual = []
generated = []
for i in range(100):
    print("current iter" + str(i))
    text = df_valid.iloc[i]["Article"]
    summary = df_valid.iloc[i]["Summary"]
    actual.append(summary)
#     print(f"Article: {text}")
#     print(f"Original Summary: {summary}")
#     print("Predicted Summary: ", end="")
    gen_sum = summarizer(text)[0]
    generated.append(gen_sum)

current iter0
current iter1
current iter2
current iter3
current iter4
current iter5
current iter6
current iter7
current iter8
current iter9
current iter10
current iter11
current iter12
current iter13
current iter14
current iter15
current iter16
current iter17
current iter18
current iter19
current iter20
current iter21
current iter22
current iter23
current iter24
current iter25
current iter26
current iter27
current iter28
current iter29
current iter30
current iter31
current iter32
current iter33
current iter34
current iter35
current iter36
current iter37
current iter38
current iter39
current iter40
current iter41
current iter42
current iter43
current iter44
current iter45
current iter46
current iter47
current iter48
current iter49
current iter50
current iter51
current iter52
current iter53
current iter54
current iter55
current iter56
current iter57
current iter58
current iter59
current iter60
current iter61
current iter62
current iter63
current iter64
current iter65
current iter66
curre

In [70]:
!pip install rouge_score
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=6845b1e2433651a02930fee96aed90642caad14952c25c1b61c919f317ff6e05
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge_score
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━

In [105]:
import evaluate
rouge = evaluate.load('rouge')
score = rouge.compute(predictions=generated, references=actual, tokenizer=lambda x: x.split())
score

{'rouge1': 0.06660116204745914,
 'rouge2': 0.016197570232582698,
 'rougeL': 0.058611493433101396,
 'rougeLsum': 0.05862394641920451}

In [72]:
import shutil
shutil.make_archive('output', 'zip', '/kaggle/working/')

'/kaggle/working/output.zip'